In [1]:
import pandas as pd 
import numpy as np
import csv
import matplotlib.pyplot as plt
import os

In [2]:
csv_files = [f for f in os.listdir('./data') if f.endswith('.csv')]
for file in csv_files:
    file_path = os.path.join('./data', file)
    df = pd.read_csv(file_path)
    print(f"File: {file}")
    print("Columns:", df.columns.tolist())
    print("First row values:", df.iloc[0].to_dict())
    print("-" * 40)

File: spiegel_rag_denazification_temporal-full-50-20_gemini.csv
Columns: ['chunk_id', 'primary_relevance_score', 'vector_similarity_score', 'llm_evaluation_score', 'score_difference', 'titel', 'datum', 'jahrgang', 'ausgabe', 'autoren', 'schlagworte', 'untertitel', 'url', 'nr_in_issue', 'time_window', 'evaluation_text', 'content_preview', 'content_length', 'full_content']
First row values: {'chunk_id': 1, 'primary_relevance_score': 1.0, 'vector_similarity_score': 0.7804124355316162, 'llm_evaluation_score': 1.0, 'score_difference': 0.2195875644683838, 'titel': 'Nach Führerprinzip', 'datum': '1951-12-25', 'jahrgang': 1951, 'ausgabe': 52, 'autoren': nan, 'schlagworte': "['Schnaps-Preise']", 'untertitel': nan, 'url': 'https://www.spiegel.de/politik/nach-fuehrerprinzip-a-7bf63df9-0002-0001-0000-000020300714?context=issue', 'nr_in_issue': nan, 'time_window': '1948-1952', 'evaluation_text': 'Score: 10.0/10 - Automatisch extrahiert', 'content_preview': 'Schon lange vorher hatten die Nationalsoz